In [ ]:
# | default_exp classes.DomoPage


In [ ]:
# | exporti
from fastcore.basics import patch_to
from dataclasses import dataclass, field

import asyncio
import httpx

import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoUser as dmdu
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.page as page_routes


In [ ]:
# | export
@dataclass
class DomoPage:
    id: str
    title: str = None
    parent_page_id: str = None
    auth: dmda.DomoAuth = field(default = None , repr = False)
    owners: list = field(default_factory=list)
    cards: list = field(default_factory=list)
    collections: list = field(default_factory=list)
    children: list = field(default_factory=list)

    def display_url(self):
        return f"https://{self.auth.domo_instance}.domo.com/page/{self.id}"

In [ ]:
# | exporti

@patch_to(DomoPage, cls_method=True)
def _from_bootstrap(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    domo_page = cls(id=dd.id, title=dd.title, auth=auth)

    if isinstance(dd.owners, list) and len(dd.owners) > 0:
        domo_page.owners = [
            dmdu.DomoUser._from_bootstrap_json(auth=auth, user_obj=user_dd)
            for user_dd in dd.owners
            if user_dd.type == 'USER'
        ]

        [print(other_dd) for other_dd in dd.owners
         if other_dd.type != 'USER']

    if isinstance(dd.children, list) and len(dd.children) > 0:
        domo_page.children = [
            cls._from_bootstrap(page_obj=child_dd, auth=auth)
            for child_dd in dd.children
            if child_dd.type == "page"
        ]

        [print(other_dd) for other_dd in dd.children
            if other_dd.type != "page"]

    return domo_page


#### sample _from_bootstrap_routes

In [ ]:
import os
import domolibrary.routes.bootstrap as bootstrap_routes

auth = dmda.DomoFullAuth(
    domo_instance='domo-community',
    domo_password = os.environ['DOJO_PASSWORD'],
    domo_username= os.environ['DOMO_USERNAME'])

res = await bootstrap_routes.get_bootstrap_pages(auth = auth)

page_obj = res.response[10]
dp = DomoPage._from_bootstrap(page_obj, auth = auth)

In [ ]:
# | exporti

@patch_to(DomoPage, cls_method=True)
async def _from_content_stacks_v3(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):
    # import domolibrary.classes.DomoCard as dc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=dd.id,
        title=dd.title,
        parent_page_id=dd.page.parentPageId,
        owners=dd.page.owners,
        collections=dd.collections,
        auth=auth
    )

    # if dd.cards and len(dd.cards) > 0:
    #     pg.cards = await asyncio.gather(
    #         *[dc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards])

    return pg


@patch_to(DomoPage, cls_method=True)
async def get_by_id(cls: DomoPage,
                    page_id: str,
                    auth: dmda.DomoAuth,
                    return_raw: bool = False, 
                    debug_api: bool = False):

    res = await page_routes.get_page_by_id(auth=auth, page_id=page_id, debug_api=debug_api)

    if return_raw:
        return res
        
    if not res.status == 200:
        return

    pg = await cls._from_content_stacks_v3(page_obj=res.response, auth=auth)

    return pg


#### sample implementation of get_from_id

In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = await DomoPage.get_by_id(page_id=page_id, auth=token_auth, return_raw=True)

domo_page.__dict__


{'status': 200,
 'response': {'id': '1761849366',
  'page': {'pageId': 1761849366,
   'ownerId': 1893952720,
   'owners': [{'id': 1893952720,
     'type': 'USER',
     'displayName': 'Jae Wilson1'}],
   'type': 'page',
   'title': 'delete me',
   'pageName': 'delete me',
   'created': 1625863915000,
   'updated': 1631900656760,
   'locked': False,
   'mobileEnabled': True,
   'sharedViewPage': True,
   'virtualPage': False,
   'isOwner': True},
  'type': 'page',
  'title': 'delete me',
  'sizes': [{'id': '2056437956', 'size': ''},
   {'id': '1548948000', 'size': ''}],
  'cards': [{'metadata': {'chartType': 'badge_vert_multibar',
     'chartVersion': '8',
     'currentLabel': '',
     'currentMethod': 'empty',
     'historyId': '6fa216c4-1465-4242-917c-3ca9e018c8be',
     'calendar': 'default',
     'columnAliases': '{}',
     'columnFormats': '{}',
     'SummaryNumberFormat': '{"type":"number","format":"#A"}'},
    'urn': '1548948000',
    'id': 1548948000,
    'type': 'kpi',
    'crea

In [ ]:
# | export
@patch_to(DomoPage)
async def get_accesslist(self,
                         auth: dmda.DomoAuth = None,
                         is_expand_users: bool = False,
                         return_raw: bool = False,
                         debug_api: bool = False):

    auth = auth or self.auth

    res = await page_routes.get_page_access_list(auth=auth,
                                                is_expand_users=is_expand_users,
                                                page_id=self.id,
                                                debug_api=debug_api
                                                )

    if return_raw:
        return res

    if not res.is_success :
        raise Exception('error getting access list')

    import domolibrary.classes.DomoUser as dmu
    import domolibrary.classes.DomoGroup as dmg
    
    tasks = await asyncio.gather( 
        dmu.DomoUsers.by_id(user_ids=[user.get('id') for user in res.response.get('users')], only_allow_one=False, auth=auth),
        * [ dmg.DomoGroup.get_by_id( group_id = group.get('id'), auth = auth) for group in res.response.get('groups')]
    )
    
    res.response.update({'users': tasks[0], 'groups': tasks[1:]})

    return res.response


#### sample get_accesslist

In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = DomoPage(id=page_id, auth=token_auth)


await domo_page.get_accesslist(is_expand_users=False, return_raw=False, debug_api=False)


{'users': [DomoUser(id='1345408759', title=None, department=None, display_name='Alexis Lorenz (DataMaven)', email_address='DataMaven42@gmail.com', role_id=1, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
  DomoUser(id='68216396', title=None, department=None, display_name='Elliott Leonard', email_address='egleonard88@gmail.com', role_id=1, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
  DomoUser(id='1893952720', title=None, department=None, display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=1, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)],
 'groups': [DomoGroup(id=746771171, name='test creator', type='adHoc', description=None),
  DomoGroup(id=1944377875, name='Test Group 2q32', type='closed', description='updated via upsert_group on 2023-04-19')],
 'resource': {'id': '1761849366', 'type': 'page'},
 'totalUserCount': 6}

In [ ]:


#     @classmethod
#     async def get_cards(cls, auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoCard as dc
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 card_list = [dc.DomoCard(id=card.get(
#                     'id'), auth=auth) for card in json.get('cards')]

#                 return card_list

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

#     async def get_datasets(auth, page_id, debug: bool = False, session: httpx.AsyncClient = None):
#         try:
#             import Library.DomoClasses.DomoDataset as dmds
#             close_session = False if session else True

#             if not session:
#                 session = httpx.AsyncClient()

#             res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug=debug, session=session)

#             if res.status == 200:
#                 json = res.response

#                 dataset_ls = [card.get('datasources')
#                               for card in json.get('cards')]

#                 return [dmds.DomoDataset(id=ds.get('dataSourceId'), auth=auth) for ds_ls in dataset_ls for ds in ds_ls]

#             else:
#                 return None

#         finally:
#             if close_session:
#                 await session.aclose()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()